# Control Line Optimizer

This notebook is a refactored version of the `Qartographer.py` script.  
It contains code cells and explanatory Markdown cells describing the purpose of each section.

---


## Required Libraries
The following Python libraries are essential for the qubit layout optimization:

- **numpy**: For efficient numerical computations and array operations
- **matplotlib**: For visualization of the qubit lattice and control lines
- **scipy.optimize**: For optimization algorithms (specifically minimize)
- **json**: For reading/writing configuration and results
- **os**: For file system operations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from scipy.optimize import minimize
import json
import os

## Physical Parameters
Basic geometric parameters for the qubit layout:

- `radius`: Defines the size of qubit representations in the visualization
- This parameter affects the visual scale of the layout but not the optimization itself

In [ ]:
# Radius of the circles representing qubits in the lattice.
radius = 1e-5

## Fixed Drive Points Configuration
This section defines the physical layout of drive points in the lattice:

- Drive points are fixed positions where control signals enter the system
- Each qubit must have a corresponding drive point
- The coordinates are specified in meters
- The array contains [x, y] coordinates for each drive point
- The layout is critical for optimal control line routing

In [ ]:
# Define the drive points in the lattice. These are now fixed.
fixed_drive_points = np.array([
[
    0.00003,
    -0.00002
],
[
    0.00013,
    -0.00002
],
[
    0.00023,
    -0.00002
],
[
    0.00033,
    -0.00002
],
[
    0.00043,
    -0.00002
],
[
    0.00053,
    -0.00002
],
[
    0.00063,
    -0.00002
],
[
    0.00073,
    -0.00002
],
[
    0.00083,
    -0.00002
],
[
    0.00093,
    -0.00002
],
[
    0.00103,
    -0.00002
],
[
    0.00113,
    -0.00002
],
[
    0.00123,
    -0.00002
],
[
    0.00133,
    -0.00002
],
[
    0.00143,
    -0.00002
],
[
    0.00153,
    -0.00002
],
[
    0.00153,
    0.00008
],
[
    0.00153,
    0.00018
],
[
    0.00153,
    0.00028
],
[
    0.00153,
    0.00038
],
[
    0.00153,
    0.00088
],
[
    0.00153,
    0.00078
],
[
    0.00153,
    0.00068
],
[
    0.00153,
    0.00058
],
[
    0.00153,
    0.00048
]
])

## Visual Parameters
Geometric constants for visualization:

- `drive_radius`: Size of drive point circles in the visualization
- `multiplexer_size`: Size of multiplexer connection points
  
These parameters are larger than qubit radius for better visibility in the plot

In [ ]:
# Radius of the circles representing drive points in the lattice.
drive_radius = 3e-5
# Size for multiplexer points (larger for visibility)
multiplexer_size = 1e-4

## Readout Line Configuration
Define the ideal readout line characteristics:

- `Ideal_readout_line_length`: Target length for qubit-to-multiplexer connections
- This parameter influences the optimization by providing a target distance
- Helps maintain uniform readout line lengths across the layout

In [ ]:
# Ideal length of the readout lines (from qubit to multiplexer), can be adjusted.
Ideal_readout_line_length = 1e-4

## Initial Multiplexer Configuration
Set up the initial multiplexer line configuration:

- `num_multiplexers`: Number of multiplexer lines in the system
- `initial_multiplexer_lines`: Initial positions for multiplexer start and end points
- Each line is defined by [[start_x, start_y], [end_x, end_y]]
- These positions will be optimized to minimize interference and improve routing

In [ ]:
# Define initial multiplexer lines (start and end points)
num_multiplexers = 3
initial_multiplexer_lines = np.array([
    [[0.0001, 0.00005], [0.0001, 0.0002]],
    [[0.0001, 0.00006], [0.0001, 0.0002]],
    [[0.0001, 0.00007], [0.0001, 0.0002]],
])

## Drive Path Resolution
Define the granularity of drive path optimization:

- `resolution`: Number of intermediate points in each drive line
- Higher resolution allows for more complex paths but increases computation time
- Each drive line will have (resolution + 2) points including start and end points
- Intermediate points are subject to optimization

In [ ]:
# Resolution for drive lines (number of intermediate points)
resolution = 1

## Optimization Parameters
Key constants that control the optimization behavior:

1. **Line Length Penalties**:
   - `GAMMA_LINE_LENGTH`: Controls the cost of long multiplexer lines, encouraging shorter routing.
   - `PENALTY_DRIVE_LENGTH`: Penalizes long drive paths to keep routing efficient.

2. **Proximity Penalties**:
   - `ALPHA_PROXIMITY_MULTI_DRIVE`: Prevents multiplexer endpoints from being placed too close to drive points.
   - `PENALTY_DRIVE_QUBIT_PROXIMITY`: Prevents drive lines from getting too close to other qubits (except their own target qubit).
   - `PENALTY_DRIVE_MULTI_PROXIMITY`: Maintains safe separation between drive lines and multiplexer lines.
   - `PENALTY_DRIVE_DRIVE_PROXIMITY`: Strongly discourages drive lines from being too close to or crossing other drive lines.

3. **Distance Thresholds**:
   - `MIN_PROXIMITY_DISTANCE`: Minimum allowed distance between components before proximity penalties are applied.
   - `EPSILON_DISTANCE`: Small constant added to distance calculations to avoid division by zero in penalty formulas.

4. **Readout Line Optimization**:
   - `DELTA_WEIGHT_READOUT_LINE`: Weight controlling how strongly the optimizer tries to match the readout line length to the ideal target.


In [ ]:
# Weight applied when penalizing multiplexer endpoints that are too close to drive points.
# Set to 0 here, effectively disabling this penalty.
ALPHA_PROXIMITY_MULTI_DRIVE = 0

# Weight applied to the squared length of each multiplexer line.
# Larger values strongly discourage long multiplexer lines.
GAMMA_LINE_LENGTH = 50000

# Weight applied to the squared difference between the actual qubit-to-multiplexer
# readout line length and the Ideal_readout_line_length.
# A high value enforces readout lines to be as close as possible to the ideal length.
DELTA_WEIGHT_READOUT_LINE = 10000000

# Weight applied to the total length of a drive line.
# Encourages shorter drive lines to reduce layout complexity.
PENALTY_DRIVE_LENGTH = 10.0

# Penalty weight for a drive line passing too close to any *other* qubit
# (excluding the qubit it is connected to).
PENALTY_DRIVE_QUBIT_PROXIMITY = 10.0

# Penalty weight for a drive line passing too close to a multiplexer line.
PENALTY_DRIVE_MULTI_PROXIMITY = 1.0

# Penalty weight for two drive lines passing too close to each other.
PENALTY_DRIVE_DRIVE_PROXIMITY = 1000.0

# The minimum allowed distance between any two components (qubits, drive lines, multiplexer lines, etc.)
# before penalties start being applied.
MIN_PROXIMITY_DISTANCE = 1e-6

# A small constant to prevent division by zero during inverse-distance penalty calculations.
# This also helps to avoid infinite penalties when distances are extremely small.
EPSILON_DISTANCE = 1e-6

# Weight applied to the total length of a drive line.
# Encourages shorter, more direct routing of drive lines.
PENALTY_DRIVE_LENGTH = 10.0

# Penalty for a drive line passing too close to any other qubit
# (other than the qubit it is directly connected to).
PENALTY_DRIVE_QUBIT_PROXIMITY = 10.0

# Penalty for a drive line routed too close to any multiplexer line.
PENALTY_DRIVE_MULTI_PROXIMITY = 1.0

# Strong penalty for drive lines routed too close to other drive lines.
# Helps avoid interference and crosstalk between drive channels.
PENALTY_DRIVE_DRIVE_PROXIMITY = 1000.0

# Minimum distance threshold between drive lines and other components
# before proximity penalties are applied.
MIN_PROXIMITY_DISTANCE = 1e-6

# A small epsilon value to prevent division from zero in proximity calculations.
EPSILON_DISTANCE = 1e-6

## Core Functions Overview

This notebook implements several key functions for qubit layout optimization:

1. **Data Management**:
   - `load_qubit_data_from_json()`: Loads qubit positions and coupling information
   - `save_optimized_layout_to_json()`: Saves the optimized layout results

2. **Geometry Calculations**:
   - `point_to_line_segment_distance()`: Computes distances between points and lines
   - `plot_lattice()`: Visualizes the complete lattice layout

3. **Optimization Functions**:
   - `optimize_multiplexer_points()`: Optimizes multiplexer line placement
   - `optimize_drive_paths()`: Optimizes paths between qubits and drive points
   - `create_initial_drive_paths()`: Generates initial path configurations

4. **Cost Functions**:
   - `multiplexer_cost_function()`: Evaluates multiplexer line placement quality
   - `drive_cost_function()`: Evaluates drive path quality

Each function is documented with its specific purpose and parameters.

In [ ]:
def load_qubit_data_from_json():
    """
    Prompts the user for a JSON filename and loads qubit coordinates and couplings.
    """
    while True:
        filename = input("Enter the JSON filename containing qubit data (e.g., optimized_qubit_layout.json): ")
        if not filename.strip():
            print("Filename cannot be empty. Please try again.")
            continue
        if not filename.endswith(".json"):
            filename += ".json"
        
        try:
            with open(filename, 'r') as f:
                data = json.load(f)
            
            # Extract qubit coordinates
            loaded_qubit_coords_dict = data.get("optimized_qubit_coordinates", {})
            loaded_qubit_coords = []
            # Sort qubits by their index (Qubit_0, Qubit_1, etc.)
            sorted_qubit_keys = sorted(loaded_qubit_coords_dict.keys(), key=lambda x: int(x.split('_')[1]))
            for key in sorted_qubit_keys:
                coord = loaded_qubit_coords_dict[key]
                loaded_qubit_coords.append([coord['x'], coord['y']])
            
            # Extract couplings
            loaded_qubit_couplings_list = data.get("optimized_couplings", [])
            loaded_qubit_couplings = []
            for coupling in loaded_qubit_couplings_list:
                loaded_qubit_couplings.append([coupling['qubit1_index'], coupling['qubit2_index']])

            print(f"\nSuccessfully loaded data from '{filename}'.")
            return np.array(loaded_qubit_coords), np.array(loaded_qubit_couplings)

        except FileNotFoundError:
            print(f"Error: File '{filename}' not found. Please check the filename and try again.")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from '{filename}'. Please ensure it's a valid JSON file.")
        except KeyError as e:
            print(f"Error: Missing expected key in JSON file: {e}. Ensure 'optimized_qubit_coordinates' and 'optimized_couplings' are present.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")


## Function: `point_to_line_segment_distance`

This helper function computes the shortest distance from a given point to a line segment,  
and also returns the exact closest point on that segment.

It is a core geometric utility used throughout the optimizer for proximity checks, including:
- Measuring how close a qubit or drive point is to a multiplexer line.
- Enforcing clearance rules to avoid routing overlaps.
- Supporting penalty calculations in the cost functions.

**Returns**:
- `distance` – The minimum Euclidean distance from the point to the segment.
- `closest_point` – The coordinates of the closest point on the segment.


In [ ]:
def point_to_line_segment_distance(p, a, b):
    p, a, b = np.array(p), np.array(a), np.array(b)
    ab = b - a
    ap = p - a
    
    ab_len_sq = np.dot(ab, ab)
    if ab_len_sq == 0:
        return np.linalg.norm(p - a), a
    
    t = np.dot(ap, ab) / ab_len_sq
    
    if t <= 0.0:
        closest_point = a
    elif t >= 1.0:
        closest_point = b
    else:
        closest_point = a + t * ab
    
    distance = np.linalg.norm(p - closest_point)
    return distance, closest_point

## Function: `plot_lattice`

This function creates a visual representation of the qubit lattice and all its key components,  
combining geometric placement data into a clear and labeled diagram.

**Purpose**:  
To provide a quick, visual validation of qubit positions, couplings, and wiring layout after optimization.

**Features**:
- **Qubits** (blue circles): Plots each qubit at its optimized coordinates.
- **Drive Points** (yellow circles): Shows fixed points where control signals are injected.
- **Qubit Couplings** (red lines): Displays logical connections between qubits.
- **Drive Lines** (green lines): Shows the optimized paths from each qubit to its drive point.
- **Multiplexer Lines** (black lines): Depicts connections between multiplexers for readout circuitry.
- **Readout Lines** (orange lines): Draws connections from each qubit to its nearest multiplexer point.

**Additional Details**:
- Handles overlapping legends by ensuring unique labels.
- Maintains equal aspect ratio for accurate geometry.
- Displays coordinate grid for reference.

**Parameters**:
- `ax_object`: Matplotlib axis object used for plotting.
- `drive_points`: Coordinates of fixed drive points.
- `drive_paths`: Optimized paths from qubits to drive points.
- `multiplexer_lines`: Start and end coordinates for multiplexer connections.
- `read_to_multi_lines`: Qubit index and its nearest point on a multiplexer line.
- `qubits`: Optimized qubit coordinates.
- `couplings`: List of qubit index pairs representing logical couplings.
- `title`: Plot title string.


In [ ]:

def plot_lattice(ax_object, drive_points, drive_paths, multiplexer_lines, read_to_multi_lines, qubits, couplings, title="Lattice Plot"):
    ax_object.clear()

    # Plotting Qubits
    for index in range(len(qubits)):
        qubit_data = qubits[index]
        circle = Circle((qubit_data[0], qubit_data[1]), radius, color='blue', fill=True, label='Qubit')
        ax_object.add_patch(circle)

    # Plotting Drive Points (Yellow Circles)
    for point_data in drive_points:
        circle = Circle((point_data[0], point_data[1]), drive_radius, color='yellow', fill=True, label='Drive Point')
        ax_object.add_patch(circle)

    # Plotting Qubit Couplings
    for coupling in couplings:
        start_qubit_index = coupling[0]
        end_qubit_index = coupling[1]

        if start_qubit_index < len(qubits) and end_qubit_index < len(qubits):
            start_coords = qubits[start_qubit_index]
            end_coords = qubits[end_qubit_index]
            ax_object.plot([start_coords[0], end_coords[0]], [start_coords[1], end_coords[1]], color='red', label='Qubit Coupling')
        else:
            print(f"Warning: Coupling {coupling} refers to out-of-bounds qubit index.")

    # Plotting Drive Lines (Green Lines)
    for path in drive_paths:
        x_coords = [p[0] for p in path]
        y_coords = [p[1] for p in path]
        ax_object.plot(x_coords, y_coords, color='green', linewidth=2, label='Drive Line')

    # Plotting Multiplexer Lines (Solid Black Lines)
    for line_info in multiplexer_lines:
        start_coords = line_info[0]
        end_coords = line_info[1]
        ax_object.plot([start_coords[0], end_coords[0]], [start_coords[1], end_coords[1]], color='black', linewidth=2, label='Multiplexer Line')

    # Plotting Readout Resonator Lines (Orange Lines)
    for line_info in read_to_multi_lines:
        qubit_index = line_info[0]
        closest_point_on_line = line_info[1]
        
        start_coords = qubits[qubit_index]
        end_coords = closest_point_on_line
        ax_object.plot([start_coords[0], end_coords[0]], [start_coords[1], end_coords[1]], color='orange', linewidth=2, label='Readout Line')

    ax_object.set_aspect('equal', adjustable='box')
    ax_object.grid(True)
    ax_object.set_xlabel("X-coordinate")
    ax_object.set_ylabel("Y-coordinate")
    ax_object.set_title(title)
    
    handles, labels = ax_object.get_legend_handles_labels()
    unique_labels = list(set(labels))
    unique_handles = [handles[labels.index(l)] for l in unique_labels]
    ax_object.legend(unique_handles, unique_labels, loc='best')


## Function: `create_initial_drive_paths`

This function generates the **initial routing paths** between qubits and their assigned drive points  
before any optimization takes place.

**Purpose**:  
To provide a simple, evenly spaced starting configuration for drive lines, which the optimizer can later refine.

**How it Works**:
1. Loops over each qubit–drive point pair.
2. Creates a direct path starting from the qubit position and ending at its fixed drive point.
3. Inserts **intermediate points** along the straight line at evenly spaced intervals, determined by the `resolution` parameter.
4. Stores both:
   - `drive_paths` – the complete path for each qubit (start → intermediate(s) → end)
   - `initial_intermediate_points` – just the intermediate waypoints, for use as optimization variables.

**Parameters**:
- `qubit_coords` *(array)*: Coordinates of all qubits.
- `fixed_drive_points` *(array)*: Coordinates of fixed drive points corresponding to each qubit.
- `resolution` *(int)*: Number of intermediate points to place between each qubit and drive point.

**Returns**:
- `drive_paths`: List of lists, where each sub-list is the full path (qubit → intermediate(s) → drive point).
- `initial_intermediate_points`: List of all intermediate points (to be optimized).


In [ ]:
def create_initial_drive_paths(qubit_coords, fixed_drive_points, resolution):
    drive_paths = []
    initial_intermediate_points = []
    for i in range(len(qubit_coords)):
        qubit_coord = qubit_coords[i]
        drive_point_coord = fixed_drive_points[i]
        path = [qubit_coord]
        
        for j in range(1, resolution + 1):
            t = j / (resolution + 1)
            intermediate_point = qubit_coord + t * (drive_point_coord - qubit_coord)
            path.append(intermediate_point)
            initial_intermediate_points.append(intermediate_point)

        path.append(drive_point_coord)
        drive_paths.append(path)
    
    return np.array(drive_paths, dtype=object), np.array(initial_intermediate_points)


## Function: `drive_cost_function`

This is the **objective function** for optimizing drive line routing.  
It assigns a numerical cost to a given layout, where a **lower cost means a better configuration**.

The optimizer adjusts the **intermediate points** along each drive path to minimize this cost.

**How the Cost is Calculated**:

1. **Path Reconstruction**  
   - Reshapes the flattened intermediate point array back into structured paths (qubit → intermediates → drive point).
   - Stores all paths for later cross-path checks.

2. **Length Penalty** (`PENALTY_DRIVE_LENGTH`)  
   - Adds a cost proportional to the total length of each drive path.
   - Encourages shorter wiring to reduce latency and potential interference.

3. **Qubit Proximity Penalty** (`PENALTY_DRIVE_QUBIT_PROXIMITY`)  
   - Adds a large cost if a drive path passes too close to *any* qubit other than its own.
   - Avoids physical and electromagnetic interference.

4. **Multiplexer Proximity Penalty** (`PENALTY_DRIVE_MULTI_PROXIMITY`)  
   - Checks each point along the drive path for closeness to multiplexer lines.
   - Helps maintain clear separation between drive lines and multiplexers.

5. **Drive-to-Drive Proximity Penalty** (`PENALTY_DRIVE_DRIVE_PROXIMITY`)  
   - Ensures that drive lines don't run too close to each other.
   - Reduces risk of crosstalk and fabrication constraints.

**Parameters**:
- `intermediate_points_flat`: Flattened array of all intermediate waypoints (optimizer variable).
- `qubit_coords_in_cost_fn`: Array of qubit coordinates.
- `fixed_drive_points_in_cost_fn`: Array of fixed drive point coordinates.
- `resolution_in_cost_fn`: Number of intermediate points between each qubit and its drive point.
- `fixed_multiplexer_lines_for_penalty`: Multiplexer line coordinates for proximity checking.

**Returns**:
- `total_cost` *(float)*: Combined penalty score — lower values are better.

---
**Optimization Goal**:  
Minimize `total_cost` to produce drive paths that are:
- Short
- Well-separated from qubits
- Well-separated from multiplexer lines
- Well-separated from other drive lines


In [ ]:
# Drive cost function with proximity and length penalties
def drive_cost_function(intermediate_points_flat, qubit_coords_in_cost_fn, fixed_drive_points_in_cost_fn, resolution_in_cost_fn, fixed_multiplexer_lines_for_penalty):
    num_qubits = len(qubit_coords_in_cost_fn)
    intermediate_points_reshaped = intermediate_points_flat.reshape(num_qubits, resolution_in_cost_fn, 2)
    total_cost = 0.0
    all_drive_paths = []

    # Calculate penalties for each drive line
    for i in range(num_qubits):
        qubit_coord = qubit_coords_in_cost_fn[i]
        drive_point_coord = fixed_drive_points_in_cost_fn[i]
        path_points = [qubit_coord] + list(intermediate_points_reshaped[i]) + [drive_point_coord]
        all_drive_paths.append(path_points)

        # Total length penalty
        drive_line_length = 0.0
        for j in range(len(path_points) - 1):
            drive_line_length += np.linalg.norm(path_points[j] - path_points[j+1])
        total_cost += PENALTY_DRIVE_LENGTH * drive_line_length

        # Proximity penalty to other qubits (not the connected one)
        for other_qubit_index in range(num_qubits):
            if other_qubit_index != i:
                other_qubit_coord = qubit_coords_in_cost_fn[other_qubit_index]
                for p_index in range(1, len(path_points) - 1): # exclude start and end points
                    point_on_drive_line = path_points[p_index]
                    dist_to_qubit = np.linalg.norm(point_on_drive_line - other_qubit_coord)
                    if dist_to_qubit < MIN_PROXIMITY_DISTANCE:
                        total_cost += PENALTY_DRIVE_QUBIT_PROXIMITY / (dist_to_qubit + EPSILON_DISTANCE)**2

        # Proximity penalty to multiplexer lines
        for multi_line in fixed_multiplexer_lines_for_penalty:
            start_multi, end_multi = multi_line
            for p_index in range(1, len(path_points)):
                p_current = path_points[p_index]
                p_previous = path_points[p_index - 1]
                
                # Simplified check for proximity to the multi-line segment
                dist_to_multi_segment, _ = point_to_line_segment_distance(p_current, start_multi, end_multi)
                if dist_to_multi_segment < MIN_PROXIMITY_DISTANCE:
                    total_cost += PENALTY_DRIVE_MULTI_PROXIMITY / (dist_to_multi_segment + EPSILON_DISTANCE)**2

    # Proximity penalty between drive lines
    for i in range(num_qubits):
        for k in range(i + 1, num_qubits):
            # Compare each point on drive line i with each point on drive line k
            for p_i in all_drive_paths[i][1:-1]: # exclude endpoints
                for p_k in all_drive_paths[k][1:-1]:
                    dist_between_paths = np.linalg.norm(p_i - p_k)
                    if dist_between_paths < MIN_PROXIMITY_DISTANCE:
                        total_cost += PENALTY_DRIVE_DRIVE_PROXIMITY / (dist_between_paths + EPSILON_DISTANCE)**2

    return total_cost

## Function: `multiplexer_cost_function`

This is the **objective function** for optimizing the placement and length of multiplexer lines.  
It evaluates a candidate configuration of multiplexer start/end points and assigns a **penalty score** —  
lower scores indicate better designs.

The optimizer changes multiplexer coordinates to minimize this score.

---

### **Cost Components**

1. **Readout Line Deviation Penalty** (`DELTA_WEIGHT_READOUT_LINE`)  
   - For each qubit, finds the *closest point* on any multiplexer line.  
   - Compares that distance to an `Ideal_readout_line_length`.  
   - Adds a penalty proportional to the square of the deviation.  
   - Encourages consistent and manufacturable readout line lengths.

2. **Multiplexer Length Penalty** (`GAMMA_LINE_LENGTH`)  
   - Adds a cost for long multiplexer lines.  
   - Promotes shorter lines to reduce signal loss and minimize routing complexity.

3. **Proximity to Drive Points Penalty** (`ALPHA_PROXIMITY_MULTI_DRIVE`)  
   - Penalizes multiplexer endpoints that are too close to any drive point.  
   - Reduces risk of interference and fabrication conflicts.

---

### **Parameters**
- `params_flat` *(ndarray)*: Flattened array of all multiplexer start/end coordinates (optimizer variable).
- `qubit_coords_for_cost` *(ndarray)*: Qubit positions for calculating readout line distances.
- `fixed_drive_points_for_penalty` *(ndarray)*: Drive point locations for proximity checking.
- `num_qubits` *(int)*: Number of qubits.
- `num_multiplexers` *(int)*: Number of multiplexer lines.

---

### **Returns**
- `total_cost` *(float)*: Total penalty value.  
  The optimizer tries to **minimize** this number to achieve optimal multiplexer routing.

---

**Optimization Goal**:  
Create multiplexer lines that are:
- Well-positioned for short and consistent readout lines.
- Minimal in total length.
- Safely distanced from drive points.


In [ ]:
def multiplexer_cost_function(params_flat, qubit_coords_for_cost, fixed_drive_points_for_penalty, num_qubits, num_multiplexers):
    multiplexer_lines = params_flat.reshape(num_multiplexers, 2, 2)
    total_cost = 0.0

    # Cost for readout line connections (qubit to closest point on closest line)
    for i in range(num_qubits):
        qubit_coord = qubit_coords_for_cost[i]
        min_dist_to_multi = float('inf')

        for j in range(num_multiplexers):
            start_point = multiplexer_lines[j, 0]
            end_point = multiplexer_lines[j, 1]
            dist, _ = point_to_line_segment_distance(qubit_coord, start_point, end_point)
            if dist < min_dist_to_multi:
                min_dist_to_multi = dist
        
        total_cost += DELTA_WEIGHT_READOUT_LINE * ((min_dist_to_multi - Ideal_readout_line_length)**2)

    # Penalty for the total length of each multiplexer line
    for j in range(num_multiplexers):
        start_point = multiplexer_lines[j, 0]
        end_point = multiplexer_lines[j, 1]
        line_length = np.linalg.norm(start_point - end_point)
        total_cost += GAMMA_LINE_LENGTH * (line_length**2)

    # Proximity penalty (multiplexer line endpoints to drive points)
    proximity_penalty_multi_drive = 0.0
    all_multi_endpoints = multiplexer_lines.reshape(-1, 2)
    for i in range(len(all_multi_endpoints)):
        for j in range(len(fixed_drive_points_for_penalty)):
            multi_endpoint = all_multi_endpoints[i]
            drive_point = fixed_drive_points_for_penalty[j]
            distance = np.linalg.norm(multi_endpoint - drive_point)
            proximity_penalty_multi_drive += ALPHA_PROXIMITY_MULTI_DRIVE / ((distance + EPSILON_DISTANCE)**2)
    total_cost += proximity_penalty_multi_drive

    return total_cost


## Function: `optimize_drive_paths`

Runs the **drive line optimization** using the `drive_cost_function` as the objective.  
It adjusts intermediate waypoints along each drive line to find the lowest-cost configuration  
while respecting proximity, length, and interference penalties.

---

### **How It Works**
1. **Flatten Initial Guess**  
   - The starting positions of all intermediate points are flattened into a 1D array so  
     the optimizer can process them.

2. **Run Optimization (`scipy.optimize.minimize`)**  
   - Uses the **L-BFGS-B** algorithm — a gradient-based, memory-efficient method  
     well-suited for high-dimensional optimization.
   - Passes all necessary geometric and layout parameters to the cost function.

3. **Reshape Optimized Points**  
   - Converts the flat result back into the `(num_qubits, resolution, 2)` format  
     for easier downstream plotting and layout reconstruction.

---

### **Parameters**
- `initial_intermediate_points` *(ndarray)*: Initial guess for intermediate waypoints.
- `qubit_coords_for_optimization` *(ndarray)*: Qubit coordinates.
- `fixed_drive_points_for_optimization` *(ndarray)*: Fixed drive point locations.
- `resolution_for_optimization` *(int)*: Number of intermediate points per drive path.
- `optimized_multiplexer_lines_for_penalty` *(ndarray)*: Final multiplexer geometry used in proximity penalty calculations.

---

### **Returns**
- *(ndarray)*: Optimized intermediate points reshaped into `(num_qubits, resolution, 2)` format.

---

**Optimization Goal**:  
Minimize drive path total cost while:
- Keeping drive lines short (`PENALTY_DRIVE_LENGTH`).
- Avoiding proximity to qubits, multiplexer lines, and other drive lines.


In [ ]:
def optimize_drive_paths(initial_intermediate_points, qubit_coords_for_optimization, fixed_drive_points_for_optimization, resolution_for_optimization, optimized_multiplexer_lines_for_penalty):
    initial_intermediate_points_flat = initial_intermediate_points.flatten()
    result = minimize(drive_cost_function, initial_intermediate_points_flat, args=(qubit_coords_for_optimization, fixed_drive_points_for_optimization, resolution_for_optimization, optimized_multiplexer_lines_for_penalty), method='L-BFGS-B')
    return result.x.reshape(-1, 2)

## Function: `optimize_multiplexer_points`

This function optimizes the placement of **multiplexer lines** to achieve the best  
balance between short, uniform readout connections and minimal interference with drive points.

It uses `multiplexer_cost_function` as the objective for optimization.

---

### **How It Works**
1. **Determine Problem Size**  
   - Calculates the number of qubits and multiplexer lines from the input data.

2. **Prepare Initial Optimization Vector**  
   - Flattens the `(num_multiplexers, 2, 2)` multiplexer coordinates into a 1D vector,  
     which is the required input format for the optimizer.

3. **Run Optimization**  
   - Calls `scipy.optimize.minimize` with:
     - **Objective function**: `multiplexer_cost_function`
     - **Method**: `L-BFGS-B` (efficient, gradient-based optimization)
     - **Goal**: Adjust multiplexer endpoints to minimize:
       - Readout line length deviation (`DELTA_WEIGHT_READOUT_LINE`)
       - Multiplexer total length (`GAMMA_LINE_LENGTH`)
       - Proximity to drive points (`ALPHA_PROXIMITY_MULTI_DRIVE`)

4. **Reshape Optimized Parameters**  
   - Converts the optimized flat vector back into `(num_multiplexers, 2, 2)` format.

5. **Assign Qubits to Closest Multiplexer Points**  
   - For each qubit:
     - Finds the nearest point on any multiplexer line segment using `point_to_line_segment_distance`.
     - Stores the `(qubit_index, closest_point_coordinates)` in `read_to_multi_lines`.

---

### **Parameters**
- `initial_multiplexer_lines` *(ndarray)*: Initial multiplexer start/end coordinates.
- `qubit_coords_for_optimization` *(ndarray)*: Positions of qubits in the layout.
- `current_drive_points` *(ndarray)*: Fixed drive point coordinates.

---

### **Returns**
1. **`optimized_multiplexer_lines`** *(ndarray)*  
   - Optimized multiplexer coordinates `(num_multiplexers, 2, 2)`.
2. **`read_to_multi_lines`** *(ndarray)*  
   - Array mapping each qubit to the closest multiplexer connection point.

---

**Optimization Goal**:  
Find multiplexer positions that:
- Keep readout lines close to their **ideal length**.
- Avoid long multiplexer traces.
- Maintain **safe distances** from drive points.


In [ ]:
def optimize_multiplexer_points(initial_multiplexer_lines, qubit_coords_for_optimization, current_drive_points):
    num_qubits = len(qubit_coords_for_optimization)
    num_multiplexers = len(initial_multiplexer_lines)
    initial_params_flat = initial_multiplexer_lines.flatten()
    result = minimize(multiplexer_cost_function, initial_params_flat,
                      args=(qubit_coords_for_optimization, current_drive_points, num_qubits, num_multiplexers),
                      method='L-BFGS-B')

    optimized_multiplexer_lines = result.x.reshape(num_multiplexers, 2, 2)

    read_to_multi_lines = []
    for i in range(num_qubits):
        qubit_coord = qubit_coords_for_optimization[i]
        min_dist_to_multi = float('inf')
        closest_point_on_line = None
        for j in range(num_multiplexers):
            start_point = optimized_multiplexer_lines[j, 0]
            end_point = optimized_multiplexer_lines[j, 1]
            dist, current_closest_point = point_to_line_segment_distance(qubit_coord, start_point, end_point)
            if dist < min_dist_to_multi:
                min_dist_to_multi = dist
                closest_point_on_line = current_closest_point
        read_to_multi_lines.append([i, closest_point_on_line])

    return optimized_multiplexer_lines, np.array(read_to_multi_lines, dtype=object)


## Function: `save_optimized_layout_to_json`

Saves the **final optimized layout** (drive points, multiplexer lines, readout connections, and drive paths)  
to a JSON file in a **fully serialized** format for later reuse, visualization, or fabrication.

---

### **How It Works**
1. **Prompt for Output Filename**
   - Asks the user for a filename to save the results.
   - Ensures the name is non-empty.
   - Appends `.json` extension automatically if not provided.

2. **Prepare Data for JSON Serialization**
   - Converts all NumPy arrays into standard Python lists (JSON-compatible).
   - Formats:
     - `drive_points` → list of `[x, y]` pairs.
     - `multiplexer_lines` → list of two endpoint coordinates per line.
     - `read_to_multi_lines` → list of `[qubit_index, closest_point_coordinates]`.
     - `drive_paths` → list of paths, each containing coordinate pairs.

3. **Assemble Data Dictionary**
   - Creates a structured dictionary with:
     - `"fixed_drive_points"`
     - `"optimized_multiplexer_lines"`
     - `"optimized_readout_to_multiplexer_lines"`
     - `"optimized_drive_paths"`

4. **Write to JSON File**
   - Uses `json.dump` with indentation for readability.
   - Prints confirmation and the full saved data for user verification.

5. **Error Handling**
   - Catches and reports:
     - File I/O errors (e.g., permission issues, invalid paths).
     - Unexpected exceptions during save.

---

### **Parameters**
- `drive_points` *(ndarray)*: Final fixed drive point coordinates.
- `multiplexer_lines` *(ndarray)*: Optimized multiplexer start/end coordinates.
- `read_to_multi_lines` *(list)*: Mapping of qubits to closest multiplexer points.
- `drive_paths` *(list)*: Complete set of optimized drive routing paths.

---

### **Output**
- **File**: JSON file containing all optimized layout data.
- **Console**: Prints a success message and a pretty-printed copy of the saved data.

---

**Purpose**:  
Ensures the optimized layout is stored in a clean, portable format  
so it can be reloaded without rerunning the optimization process.


In [ ]:
def save_optimized_layout_to_json(drive_points, multiplexer_lines, read_to_multi_lines, drive_paths):
    while True:
        filename = input("Enter the desired filename for the optimized layout data (e.g., optimized_layout_details.json): ")
        if not filename.strip():
            print("Filename cannot be empty. Please try again.")
            continue
        if not filename.endswith(".json"):
            filename += ".json"
        break

    # Convert NumPy arrays to lists for JSON serialization
    formatted_drive_points = drive_points.tolist()
    formatted_multiplexer_lines = multiplexer_lines.tolist()
    
    # read_to_multi_lines contains qubit index and a coordinate.
    formatted_read_to_multi_lines = []
    for line_info in read_to_multi_lines:
        formatted_read_to_multi_lines.append([int(line_info[0]), line_info[1].tolist()])

    # drive_paths is a list of lists of arrays, need to convert inner arrays to lists
    formatted_drive_paths = []
    for path in drive_paths:
        formatted_path = [point.tolist() for point in path]
        formatted_drive_paths.append(formatted_path)

    data_to_save = {
        "fixed_drive_points": formatted_drive_points,
        "optimized_multiplexer_lines": formatted_multiplexer_lines,
        "optimized_readout_to_multiplexer_lines": formatted_read_to_multi_lines,
        "optimized_drive_paths": formatted_drive_paths
    }

    try:
        with open(filename, 'w') as f:
            json.dump(data_to_save, f, indent=4)
        print(f"\nSuccessfully saved optimized layout data to '{filename}'.")
        print("-" * 50)
        print(json.dumps(data_to_save, indent=4))
        print("-" * 50)
    except IOError as e:
        print(f"Error: Could not write to file '{filename}'. Reason: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while saving JSON: {e}")


## Main Optimization Workflow

This is the **core execution sequence** of the optimizer, coordinating all major steps  
from loading data to saving the final optimized layout.

---

### **Steps in the Workflow**

1. **Load Input Data**
   - Calls `load_qubit_data_from_json()` to read qubit coordinates and couplings from a user-specified JSON file.

2. **Validate Drive Point Count**
   - Ensures the number of `fixed_drive_points` matches the number of qubits.
   - If too many, truncates the list.
   - If too few, warns that some qubits will not have an assigned drive point.

3. **Optimize Multiplexer Lines**
   - Runs `optimize_multiplexer_points()` to adjust multiplexer positions for minimal cost.
   - Prints the resulting optimized multiplexer coordinates.

4. **Initialize Drive Paths**
   - Uses `create_initial_drive_paths()` to generate the starting routes from each qubit to its drive point.

5. **Optimize Drive Paths**
   - If intermediate points exist:
     - Runs `optimize_drive_paths()` to refine routing based on proximity and length penalties.
     - Reshapes results into full path definitions.
   - If no points exist, skips optimization.

6. **Plot the Optimized Lattice**
   - Calls `plot_lattice()` to visualize:
     - Qubit positions
     - Drive lines
     - Multiplexer lines
     - Readout resonator connections

7. **Save Results**
   - Invokes `save_optimized_layout_to_json()` to store all optimized data  
     (drive points, multiplexer lines, readout connections, and drive paths) in a JSON file.

---

**Purpose**:  
This workflow ties together **data loading, validation, optimization, visualization, and saving**  
into a complete layout optimization pipeline.


In [ ]:

# Load qubit data from JSON file
qubit_coords, qubit_couplings = load_qubit_data_from_json()

# Ensure the number of fixed drive points matches the number of qubits loaded
if len(fixed_drive_points) != len(qubit_coords):
    print(f"Warning: Number of fixed_drive_points ({len(fixed_drive_points)}) does not match number of qubits loaded ({len(qubit_coords)}). This might lead to unexpected behavior.")
    if len(fixed_drive_points) > len(qubit_coords):
        fixed_drive_points = fixed_drive_points[:len(qubit_coords)]
        print(f"Truncated fixed_drive_points to {len(fixed_drive_points)}.")
    else:
        print(f"Not enough fixed_drive_points for all qubits. Some qubits might not have a corresponding drive point.")


drive_points_fixed = fixed_drive_points

print("--- Starting Optimization for Multiplexer Lines ---")
optimized_multiplexer_lines, optimized_read_to_multi_lines = \
    optimize_multiplexer_points(initial_multiplexer_lines, qubit_coords, drive_points_fixed)
print("Optimized Multiplexer Lines:\n", optimized_multiplexer_lines)

print("\n--- Starting Optimization for Drive Paths ---")
initial_drive_paths, initial_intermediate_points = create_initial_drive_paths(qubit_coords, drive_points_fixed, resolution)

# Check if initial_intermediate_points is empty, which can happen if qubit_coords is empty
if initial_intermediate_points.size == 0:
    print("No intermediate points to optimize. Skipping drive path optimization.")
    optimized_intermediate_points = np.array([])
    optimized_drive_paths = []
else:
    optimized_intermediate_points = optimize_drive_paths(
        initial_intermediate_points,
        qubit_coords,
        drive_points_fixed,
        resolution,
        optimized_multiplexer_lines
    )

    # Reconstruct the optimized drive paths for plotting
    optimized_drive_paths = []
    optimized_intermediate_points_reshaped = optimized_intermediate_points.reshape(len(qubit_coords), resolution, 2)
    for i in range(len(qubit_coords)):
        path = [qubit_coords[i]] + list(optimized_intermediate_points_reshaped[i]) + [drive_points_fixed[i]]
        optimized_drive_paths.append(path)
    print("Optimized Intermediate Drive Points (reshaped):\n", optimized_intermediate_points_reshaped)

# Plot the optimized lattice
fig_optimized, ax_optimized = plt.subplots(figsize=(10, 8))
plot_lattice(ax_optimized, drive_points_fixed, optimized_drive_paths,
              optimized_multiplexer_lines, optimized_read_to_multi_lines,
              qubit_coords, qubit_couplings, title=f"Optimized Lattice (Multiplexer First)")
plt.show()

# Save the optimized layout data to a JSON file
save_optimized_layout_to_json(
    drive_points_fixed,
    optimized_multiplexer_lines,
    optimized_read_to_multi_lines,
    optimized_drive_paths
)